In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
spotify = pd.read_csv('/kaggle/input/spotify-dataset-19212020-160k-tracks/data.csv')
spotify.head()

In [ ]:
print(len(spotify))

In [ ]:
import seaborn as sns

# ВСЯКАЯ ЗАБАВНАЯ ЕРУНДА

**первая удивительная вещь - в мажоре песня написана или в миноре очень незначительно влияет на то, насколько она радостная**

In [ ]:
ax = sns.barplot(x="mode", y="valence", data=spotify)
ax.set()

**зато у песен в миноре гораздо чаще матерные тексты**

In [ ]:
ax = sns.barplot(x="mode", y="explicit", data=spotify)
ax.set()

**так что музыкалка лжет - мажор и минор это не про весело и грустно, это про то, хорошо ли туда вписывается мат**

# Можно отличать исполнителей по данным, предложенным спотифаем?

для того, чтобы ответить на этот вопрос, я выберу двух похожих исполнителей, которые были активны в примерно один и тот же промежуток времени - the beatles и the rolling stones. очевидно, что если сравнивать шопена и сердючку сразу все будет понятно по danceability и speechiness - так что совсем разных выбирать неинтересно. интересно, получится ли по этому ограниченному набору показателей отличить кого-то похожего

In [ ]:
beatles_n_stones = spotify.loc[lambda spotify: (spotify['artists'] == "['The Beatles']")|(spotify['artists'] == "['The Rolling Stones']")]
beatles_n_stones = beatles_n_stones[beatles_n_stones.year<1970]

перестраховываюсь потому что не помню как эти поцаны копируются нормально или нет

In [ ]:
import copy
checkpoint_bns = copy.deepcopy(beatles_n_stones)

это маленький бесполезный чекпоинт для моего личного успокоения

In [ ]:
print(len(beatles_n_stones), 275+263, sep = ' ')

In [ ]:
beatles_n_stones.tail()

отфигачиваем от матрицы все что деревья гипотетически могут запомнить по значению и данные не в числах

In [ ]:
del beatles_n_stones['id']
del beatles_n_stones ['release_date']
del beatles_n_stones ['year']
del beatles_n_stones ['name']
beatles_n_stones = beatles_n_stones.iloc[:, 1:] #дада мне лень вспоминать как называется тот столбец


пусть beatles = 1, а stones = 0
заменим значения в таблице и заодно создадим список который это запомнит потому что у меня башка дырявая

In [ ]:
beatles_n_stones = beatles_n_stones.replace("['The Beatles']", 1)
beatles_n_stones = beatles_n_stones.replace("['The Rolling Stones']", 0)

In [ ]:
B_n_S = ['Rolling Stones', 'Beatles']

теперь разделим это на X и Y а потом на тестовую и учебную выборки

In [ ]:
Y = beatles_n_stones.iloc[:, 1].values
del beatles_n_stones ['artists']
X = beatles_n_stones.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

кусо4ек лежащий тут пока что бесполезен, но может понадобиться потом, когда я буду искать в каких именно песнях классификатор ошибся. да, я наивно надеюсь на то, что у всех песен разная длительность и делаю словарь который по номеру песни в тестовом иксе выдает ее длительность как значение

ТУТ КСТАТИ ОЧЕНЬ ПРИНЦИПИАЛЬНЫЙ МОМЕНТ - ЗАПУСКАТЬ ЕГО НАДО СТРОГО ПОСЛЕ ПРОШЛОГО КУСКА КОДА, ПОКА ВСЕ ЗНАЧЕНИЯ НЕ НОРМАЛИЗОВАНЫ

In [ ]:
#print(X_test[1])
finding_the_name = {i:X_test[i, 2] for i in range(len(X_test))}
finding_the_name

вроде как нормализовать все данные при форесте не очень нужно, но вот я при нужде умею смотрите

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from matplotlib import pyplot

# Обучение и исследования вклада фич (спойлер: селекшна не будет)

я обучаю свой лес на 200 деревьях. я и 20 пробовала, и 1000, но 200 дает лучшую balanced accuracy и скорость, так что я 200 оставила
снизу я пишу кривой код по показыванию значимости фич. 

анализ - тебе

In [ ]:
from sklearn.model_selection import cross_validate

вот я сначала хотела сделать 10fold cross validation потому что читала об этом в статье а потом передумала потому что не умею и у меня голова болит. как-нибудь в другой раз

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=200)#, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

my_features = list(beatles_n_stones)
importance = classifier.feature_importances_
for i,v in enumerate(importance):
    print('Feature: %0s, Score: %.5f' % (my_features[i],v))

pyplot.bar([my_features[x][:3] for x in range(len(importance))], importance)
pyplot.show()


алгоритм отличает the rolling stones от the beatles по популярности, энергичности,продолжительности и количеству инструментала в песне - эти параметры наиболее значимы

самыми бесполезными фичами оказались наличие мата и тональность песни (ака совокупность бесполезных фич показывающих тонику и лад - тут только ионийский и дорийский). с одной стороны это странно, потому что мы видели, как битлз тяготеют к до мажору, а с другой более продвинутые музыкально ролинги тоже вполне себе могли использовать тот же до мажор..........

вообще жалко что в датасете были только эолийский и ионийский (минор и мажор) - если б были другие, можно было бы отследить отклонения в джаз и всякие эксперименты

*нет мы не видели что битлз тяготеют к до мажору потому что я это убрала. ну ничего - тогда сюда перепишу, чтобы наглядно было. тут на каунтплоте отмечены тональности - тоники цифрами (0 - до), а лад ноликом или единичкой. короче чаще всего до мадор, на втором месте ре мажор, потом соль мажор - те тональности, в которых для трезвучий от тоники, доминанты и субдоминанты мало сложных барре для гитары*

In [ ]:
beatles = spotify.loc[lambda spotify: (spotify['artists'] == "['The Beatles']")]
ax = sns.countplot(x="key", hue="mode", data=beatles)

*и еще оффтопом: битлы с годами грустнели*

In [ ]:
ax = sns.barplot(x="year", y="valence",palette="rocket",  data=beatles.loc[beatles['year']<1970])

sns.set(rc={'figure.figsize':(8,3)})

# Результаты предсказания

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

не вах так точно но под пивас пойдет

## Визуализируем это на ROC-кривой и Presision-Recall

In [ ]:
from sklearn.metrics import plot_precision_recall_curve
disp = plot_precision_recall_curve(classifier, X_test, y_test)

In [ ]:
from sklearn.metrics import plot_roc_curve
disp = plot_roc_curve(classifier, X_test, y_test)

делаю вид что знаю что значат эти графики а на самом деле меня так в 10 классе научили вот пользуюсь как могу выжимаю из своей информатики все

шучу я знаю что они значат но понимаю их ток на уровне "ROC-кривая обозначает точность и должна прилегать к правому верхнему углу а пресижн реколл к левому верхнему"

## В каких же песнях классификатор ошибся?

In [ ]:
for row_index, (input, prediction, label) in enumerate(zip (X_test, y_pred, y_test)):
  if prediction != label:
    length = finding_the_name[row_index]
    name = checkpoint_bns.loc[checkpoint_bns['duration_ms'] == length].iloc[0, 13]
    print(name, 'был классифицирован как', B_n_S[prediction], 'а должен был быть классифицирован как', B_n_S[label])

резы очень прикольные, потому что, порывшись на всяких форумах "какая песня стоунз похожа на песни битлз" (форумов на обратную тему оч мало и они невнятные, что логично) я нашла совпадения: многие говорят, что Come On, Ruby Tuesday, Under My Thumb, [Paint It Black](http://www.quora.com/What-songs-by-The-Rolling-Stones-come-the-closest-to-sounding-like-material-from-The-Beatles) и It's All Too Much(из-за использование ситар) похожи на  [битлов](https://www.reddit.com/r/beatles/comments/7gycmf/which_rolling_stones_song_sounds_like_beatles_one/) и они при прогонке алгоритма оказываются в неправильных предсказаниях.

про ruby tuesday вообще общеизвестный факт

дада всего неправильных предсказаний не так много и учитывая то что они действительно (по словам фанатов) похожи на правильные предсказания, крутой классификатор получился